In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])


@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(cluster,client)
# client.close()
# http://localhost:8787/status


LocalCluster(3334f920, 'tcp://127.0.0.1:44405', workers=4, threads=16, memory=24.59 GiB) <Client: 'tcp://127.0.0.1:44405' processes=4 threads=16, memory=24.59 GiB>


distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-oqeisxip', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-1cx41uhr', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-1ihxbx9q', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-dl4k54xi', purging


In [2]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = f'{crop[key]}'
    for disease_code in value:
        label = f'{disease_code}'
        disease_label_description[label] = f'{disease_code}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{key}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [3]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [4]:
#making_csv_set
row_csv = dd.read_csv(path+"/train/*/*.csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})
partitions = row_csv.to_delayed()
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
X_train = np.array(dask.compute(*datas))


distributed.utils - ERROR - 'start'
Traceback (most recent call last):
  File "/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/distributed/utils.py", line 681, in log_errors
    yield
  File "/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/distributed/dashboard/components/shared.py", line 285, in update
    ts = metadata["keys"][self.key]
KeyError: 'start'
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7f12ba384950>>, <Task finished coro=<_needs_document_lock.<locals>._needs_document_lock_wrapper() done, defined at /home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/bokeh/server/session.py:77> exception=KeyError('start')>)
Traceback (most recent call last):
  File "/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/home/gob/anacon

멀티라벨

In [5]:
label_list = sorted(glob(path+"/train/*/*.json"))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_dc = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)

In [6]:
label.shape,label_crop.shape,label_dc.shape,label_risk.shape, X_train.shape

((5767,), (5767,), (5767,), (5767,), (5767, 290, 9))

In [30]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }
models = []
for i in range(len(csv_features)):
  start = time.time()
  # globals()["XGB_g_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label)
  _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label)
  models.append(_)
  # globals()["XGB_crop_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_crop)
  _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_crop)
  models.append(_)
  # globals()["XGB_dc_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_dc)
  _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_dc)
  models.append(_)
  # globals()["XGB_risk_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_risk)
  _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label_risk)
  models.append(_)
  print(time.time()-start)

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


16.547953367233276
15.158648490905762
15.655789375305176
17.159715175628662
16.34363031387329
16.45718026161194
14.51174807548523
15.450153827667236
14.836581707000732


모델학습

In [38]:
models[4].predict_proba(X_train[:,:,i]).shape

(5767, 25)

In [41]:
for i in range(4):print(i)

0
1
2
3


In [58]:
def process_x(dataset,models):
    stacked_data = []

    for i in range(len(csv_features)):
        for j in range(4):
            _ = models[i*4 + j].predict_proba(dataset[:,:,i])
            stacked_data.append(_)
            

    # x0=XGB_0.predict(dataset[:,:,0])
    # x1=XGB_1.predict(dataset[:,:,1])
    # x2=XGB_2.predict(dataset[:,:,2])
    # x3=XGB_3.predict(dataset[:,:,3])
    # x4=XGB_4.predict(dataset[:,:,4])
    # x5=XGB_5.predict(dataset[:,:,5])
    # x6=XGB_6.predict(dataset[:,:,6])
    # x7=XGB_7.predict(dataset[:,:,7])
    # x8=XGB_8.predict(dataset[:,:,8])
    # arr_x = np.array([x0,x1,x2,x3,x4,x5,x6,x7,x8,]).T
    result = np.concatenate(stacked_data,axis=1)
    return result

In [64]:
processed_x = process_x(X_train,models)

In [65]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_G = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(processed_x,label)
print(time.time()-start)

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


9.303757190704346


평가

In [66]:
def csv_path(label):
    # csv_path = f'{file}\\{file_name}.csv'
    # l_path = os.getcwd() + '\\data\\test\\%s\\%s.csv'%{label,label}
    csv_path = os.getcwd() +f'/test/{label}/{label}.csv'
    return csv_path
    
def img_path(label):
    # csv_path = f'{file}\\{file_name}.csv'
    # l_path = os.getcwd() + '\\data\\test\\%s\\%s.csv'%{label,label}
    img_path = os.getcwd() +f'/test/{label}/{label}.jpg'
    return img_path

In [67]:
#making_csv_set
test_csv = dd.read_csv(path+"/test/*/*.csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
    '내부 이슬점 최저': 'object',
    '내부 이슬점 평균': 'object'})
partitions = test_csv.to_delayed()
datas_test = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
# X_train = np.array(dask.compute(*datas_test))

In [70]:
submission = pd.read_csv(path+"/sample_submission.csv")
# submission['label'] = preds
minibatch_size = 500
data_length = len(submission)
loop_num = int(np.floor(data_length/minibatch_size))

# for i 
# xgb.fit(x_prime_train,y_train,xgb_model = xgb)
count = 0
for i in range(loop_num):
# for i in range(2):
    print(i)
    if i !=loop_num-1:
        
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        
        #예측
        pred_raw = XGB_G.predict(process_x(np.array(dask.compute(*datas_test[start_p:end_p])),models))
        submission.loc[start_p:end_p-1,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
        
    else:
        start_p = count*minibatch_size
       
        #예측
        pred_raw = XGB_G.predict(process_x(np.array(dask.compute(*datas_test[start_p:])),models))
        submission.loc[start_p:,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
    
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [ ]:
submission.to_csv(path+'/submissions/gogo_submission_07.csv', index=False)